In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r"D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\online_retail_2010_2011.csv", encoding='latin-1')

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns")
df.info()
df.describe()

Dataset contains 541910 rows and 8 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


,Quantity,Price,Customer ID
count,541910.000000,541910.000000,406830.000000
mean,9.552234,4.611138,15287.684160
std,218.080957,96.759765,1713.603074
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [5]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
# Checking duplicate values

df.duplicated().sum()

5268

In [9]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [11]:
# 1a. Handle missing values (especially CustomerID and Description)

df = df.dropna(subset=['Customer ID', 'Description'])

In [13]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [15]:
# Convert Customer ID to integer

df['Customer ID'] = df['Customer ID'].astype(int)

In [ ]:
df.info()

In [19]:
# 1b. Remove duplicate rows (keep first)

df = df.drop_duplicates(keep='first').reset_index(drop=True)

In [21]:
print(f"Dataset contains {df.shape[0]} rows and {df.shape[1]} columns")

Dataset contains 401605 rows and 8 columns


In [23]:
df.duplicated().sum()

0

In [25]:
# 1c. Handle canceled orders (where InvoiceNo starts with 'C')

# Ensure Invoice is string
df['Invoice'] = df['Invoice'].astype(str)

# Rows where Invoice starts with 'C' (case-sensitive usually)
mask_cancel = df['Invoice'].str.startswith('C')

mask_cancel.sum()

8872

In [27]:
df = df[~mask_cancel].reset_index(drop=True)

df.shape

(392733, 8)

In [ ]:
# 1d. Remove or cap outliers in Quantity and UnitPrice

display(df.sort_values('Quantity', ascending=False).head(10))
display(df.sort_values('Price', ascending=False).head(10))

In [31]:
q_low = df['Quantity'].quantile(0.01)
q_high = df['Quantity'].quantile(0.99)

p_low = df['Price'].quantile(0.01)
p_high = df['Price'].quantile(0.99)

print("Quantity cutoffs:", q_low, q_high)
print("Price cutoffs:", p_low, p_high)

Quantity cutoffs: 1.0 120.0
Price cutoffs: 0.21 14.95


In [33]:
mask_keep = (
    (df['Quantity'] >= q_low) & (df['Quantity'] <= q_high) &
    (df['Price'] >= p_low) & (df['Price'] <= p_high)
)


df_trimmed = df[mask_keep].reset_index(drop=True)

print("Shape before:", df.shape, "after trimming:", df_trimmed.shape)

Shape before: (392733, 8) after trimming: (382280, 8)


In [35]:
# 1e. Convert InvoiceDate to proper datetime format

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check parsing success
print("Null dates after parsing:", df['InvoiceDate'].isna().sum())
print(df['InvoiceDate'].dtype)
display(df[['Invoice','InvoiceDate']].head())

Null dates after parsing: 0
datetime64[ns]


,Invoice,InvoiceDate
0,536365,2010-12-01 08:26:00
1,536365,2010-12-01 08:26:00
2,536365,2010-12-01 08:26:00
3,536365,2010-12-01 08:26:00
4,536365,2010-12-01 08:26:00


In [37]:
# EDA - 2a.  How many unique products are there?

count_products = df['Description'].value_counts()

unique_products = count_products.count()

print(f'No of Unique products = {unique_products}')

No of Unique products = 3877


In [39]:
# EDA - 2a.  How many unique products are there?

unique_products = df['Description'].nunique()
print(f'No of Unique products = {unique_products}')

No of Unique products = 3877


In [41]:
# EDA - 2b.  What is the total number of transactions?

transactions = df['Invoice'].value_counts()

no_of_transactions = transactions.count()

print(f'No of Transactions = {no_of_transactions}')

No of Transactions = 18536


In [43]:
# EDA - 2b.  What is the total number of transactions?

no_of_transactions = df['Invoice'].nunique()
print(f'No of Transactions = {no_of_transactions}')

No of Transactions = 18536


In [45]:
# EDA - 2c.  How many unique customers are there?

unique_customers = df['Customer ID'].nunique()

print(f'No of Unique Customers = {unique_customers}')

No of Unique Customers = 4339


In [47]:
# EDA - 2d.  What countries do the customers belong to?

countries = df['Country'].unique()

print(f'Countries = {countries}')

Countries = ['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


In [49]:
# Product Analysis - 3a.  What are the top 10 selling products by quantity?

print(f'The top 10 selling products are given below:\n\n{count_products.head(10)}')

The top 10 selling products are given below:

Description
WHITE HANGING HEART T-LIGHT HOLDER    2016
REGENCY CAKESTAND 3 TIER              1714
JUMBO BAG RED RETROSPOT               1615
ASSORTED COLOUR BIRD ORNAMENT         1395
PARTY BUNTING                         1390
LUNCH BAG RED RETROSPOT               1303
SET OF 3 CAKE TINS PANTRY DESIGN      1152
POSTAGE                               1100
LUNCH BAG  BLACK SKULL.               1078
PACK OF 72 RETROSPOT CAKE CASES       1050
Name: count, dtype: int64


In [51]:
# Product Analysis - 3b.  What are the top 10 revenue-generating products?

df['Revenue'] = df['Quantity'] * df['Price']

revenue_by_product = df.groupby('Description')['Revenue'].sum()

top_10_revenue_products = revenue_by_product.sort_values(ascending=False).head(10)

print(f'The top 10 revenue-generating products are given below:\n\n{top_10_revenue_products}')

The top 10 revenue-generating products are given below:

Description
PAPER CRAFT , LITTLE BIRDIE           168469.60
REGENCY CAKESTAND 3 TIER              142264.75
WHITE HANGING HEART T-LIGHT HOLDER    100392.10
JUMBO BAG RED RETROSPOT                85040.54
MEDIUM CERAMIC TOP STORAGE JAR         81416.73
POSTAGE                                77821.96
PARTY BUNTING                          68785.23
ASSORTED COLOUR BIRD ORNAMENT          56413.03
Manual                                 53419.93
RABBIT NIGHT LIGHT                     51251.24
Name: Revenue, dtype: float64


In [53]:
# Product Analysis - 3c.  Which products have negative or zero unit prices?

invalid_price_products = df[df['Price'] <= 0]

invalid_products_list = invalid_price_products['Description'].unique()

print(f'Products having invalid prices are given below:\n\n{invalid_products_list}')

Products having invalid prices are given below:

['ROUND CAKE TIN VINTAGE GREEN' 'ADVENT CALENDAR GINGHAM SACK'
 'REGENCY CAKESTAND 3 TIER' 'PAPER BUNTING RETROSPOT'
 'PLASTERS IN TIN SKULLS' 'ORGANISER WOOD ANTIQUE WHITE '
 'FAIRY CAKES NOTEBOOK A6 SIZE' 'CERAMIC BOWL WITH LOVE HEART DESIGN'
 'MINI CAKE STAND  HANGING STRAWBERY' 'HEART GARLAND RUSTIC PADDED'
 'CHILDS BREAKFAST SET CIRCUS PARADE' 'PARTY BUNTING'
 'SET OF 6 SOLDIER SKITTLES' ' OVAL WALL MIRROR DIAMANTE '
 'JAM MAKING SET WITH JARS' 'Manual' 'SET OF 6 NATIVITY MAGNETS '
 'SET OF 2 CERAMIC PAINTED HEARTS ' 'SET OF 2 CERAMIC CHRISTMAS REINDEER'
 '36 FOIL STAR CAKE CASES ' 'POLKADOT RAIN HAT '
 'PADS TO MATCH ALL CUSHIONS' 'GLASS CLOCHE SMALL'
 'PASTEL COLOUR HONEYCOMB FAN' 'BISCUIT TIN VINTAGE CHRISTMAS'
 'CHRISTMAS PUDDING TRINKET POT ' 'JUMBO BAG SPACEBOY DESIGN'
 'RED KITCHEN SCALES' "CHILDREN'S APRON DOLLY GIRL "
 'MINI LIGHTS WOODLAND MUSHROOMS' 'SET OF 9 BLACK SKULL BALLOONS'
 'PAPER BUNTING VINTAGE PAISLEY' 'HANGING

In [55]:
# Customer Analysis: - 4a.  Which customers bought the most products?

top_buyers = df['Customer ID'].value_counts().head(10)

print(f'Top 10 customers who bought the most products are given below:\n\n{top_buyers}')

Top 10 customers who bought the most products are given below:

Customer ID
17841    7676
14911    5672
14096    5111
12748    4413
14606    2677
15311    2366
14646    2080
13089    1814
13263    1667
14298    1637
Name: count, dtype: int64


In [ ]:
# Customer Analysis: - 4b.  What is the distribution of purchases per customer?

purchases_per_customer_rows = df['Customer ID'].value_counts()



plt.figure(figsize=(15,5))
sns.histplot(purchases_per_customer_rows, bins=150, kde=True)
plt.title('Distribution of Purchase Entries per Customer')
plt.xlabel('Number of Line Items')
plt.ylabel('Number of Customers')
plt.show()

In [59]:
# Customer Analysis: - 4c.  Are there any loyal customers?

customer_purchases = df.groupby('Customer ID')['Invoice'].nunique()

frequent_buyers = customer_purchases.sort_values(ascending=False).head(10)

print(f'Top 10 Frequent Buyers(Loyal Customers) are given below:\n\n{frequent_buyers}')

Top 10 Frequent Buyers(Loyal Customers) are given below:

Customer ID
12748    210
14911    201
17841    124
13089     97
14606     93
15311     91
12971     86
14646     74
16029     63
13408     62
Name: Invoice, dtype: int64


In [61]:
# Time Series & Seasonal Trends: - 5a.  What are the monthly sales trends?

df['Month'] = df['InvoiceDate'].dt.to_period('M')

monthly_sales = df.groupby('Month')['Revenue'].sum()

monthly_sales

Month
2010-12     570422.730
2011-01     568101.310
2011-02     446084.920
2011-03     594081.760
2011-04     468374.331
2011-05     677355.150
2011-06     660046.050
2011-07     598962.901
2011-08     644051.040
2011-09     950690.202
2011-10    1035642.450
2011-11    1156205.610
2011-12     517208.440
Freq: M, Name: Revenue, dtype: float64

In [ ]:
plt.figure(figsize=(12,5))
monthly_sales.plot(kind='line', marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.show()

In [65]:
# Time Series & Seasonal Trends: - 5b.  What are the daily or weekly patterns of transactions?

daily_transactions = df.groupby(df['InvoiceDate'].dt.date)['Invoice'].nunique()

daily_transactions

InvoiceDate
2010-12-01    121
2010-12-02    137
2010-12-03     57
2010-12-05     87
2010-12-06     94
             ... 
2011-12-05    116
2011-12-06    110
2011-12-07    104
2011-12-08    113
2011-12-09     41
Name: Invoice, Length: 305, dtype: int64

In [ ]:
weekly_transactions = df.groupby(df['InvoiceDate'].dt.isocalendar().week)['Invoice'].nunique()

weekly_transactions

In [ ]:
plt.figure(figsize=(12,5))
daily_transactions.plot(kind='line')
plt.title('Daily Transaction Trend')
plt.xlabel('Date')
plt.ylabel('Number of Transactions')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
weekly_transactions.plot(kind='line', marker='o')
plt.title('Weekly Transaction Trend')
plt.xlabel('Week Number')
plt.ylabel('Number of Transactions')
plt.grid(True)
plt.show()

In [73]:
# Time Series & Seasonal Trends: - 5c.  Identify peak sales months and slow seasons.

sorted_monthly_sales = monthly_sales.sort_values(ascending=False)

sorted_monthly_sales.head(3)
sorted_monthly_sales.tail(3)

Month
2011-12    517208.440
2011-04    468374.331
2011-02    446084.920
Freq: M, Name: Revenue, dtype: float64

In [ ]:
plt.figure(figsize=(10,5))
sorted_monthly_sales.plot(kind='bar')
plt.title('Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()

In [77]:
# Country Analysis: - 6a.  Which countries have the highest number of orders?

highest_countries = df['Country'].value_counts().sort_values(ascending=False).head(10)


print(f'Top 10 Countries who bought the most products are given below:\n\n{highest_countries}')

Top 10 Countries who bought the most products are given below:

Country
United Kingdom    349227
Germany             9027
France              8328
EIRE                7228
Spain               2480
Netherlands         2363
Belgium             2031
Switzerland         1842
Portugal            1453
Australia           1184
Name: count, dtype: int64


In [ ]:
highest_countries.plot(kind='bar', figsize=(10,5), title='Top 10 Countries by Purchase Rows')
plt.ylabel('Number of Purchase Entries')
plt.show()

In [81]:
# Country Analysis: - 6b.  Revenue comparison across countries

revenue_countries = df.groupby('Country')['Revenue'].sum()

top_revenue_countries = revenue_countries.sort_values(ascending=False).head(10)

top_revenue_countries

Country
United Kingdom    7285024.644
Netherlands        285446.340
EIRE               265262.460
Germany            228678.400
France             208952.310
Australia          138453.810
Spain               61558.560
Switzerland         56443.950
Belgium             41196.340
Sweden              38367.830
Name: Revenue, dtype: float64

In [ ]:
plt.figure(figsize=(10,5))
top_revenue_countries.plot(kind='bar')
plt.title('Top 10 Countries by Revenue')
plt.xlabel('Country')
plt.ylabel('Total Revenue')
plt.show()

In [ ]:
# Below is the  Working for POWER BI:

In [87]:
df.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\cleaned_retail_data.csv', index=False)

In [101]:
monthly_sales_trend_csv  = sorted_monthly_sales.sort_index()

monthly_sales_trend_csv.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\monthly_sales_trend.csv')

In [99]:
top_10_products_qty_csv = count_products.head(10)

top_10_products_qty_csv.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\top_10_products_qty.csv')

In [103]:
top_10_products_revenue_csv = top_10_revenue_products

top_10_products_revenue_csv.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\top_10_products_revenue.csv')

In [105]:
loyal_customers_csv = frequent_buyers

loyal_customers_csv.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\loyal_customers.csv')

In [107]:
country_revenue_table_csv = top_revenue_countries

country_revenue_table_csv.to_csv(r'D:\Farooq_Hussain\Python\Data Analysis\07 - EDA_Online_Retail_Sales\data\country_revenue_table.csv')